In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DL_final/

# 顯示所有行列
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)


# 匯出成csv
def output(filepath, data):
    df_SAMPLE = pd.DataFrame.from_dict(data)
    df_SAMPLE.to_csv(filepath, index=False)
    print('Success output to ' + filepath)


# -----------------------------------------------------合併資料-----------------------------------------------------
# 約執行5分鐘
def merge():
    # read csv files
    df_report = pd.read_csv('/content/drive/MyDrive/DL_final/data/report.csv', header=0, low_memory=False,
                            names=["ID", "資料年度", "資料月份", "酪農場代號", "乳牛編號",
                                   "父親牛精液編號", "母親乳牛編號", "出生日期", "胎次", "泌乳天數",
                                   "乳量", "最近分娩日期", "採樣日期", "月齡", "檢測日期",
                                   "最後配種日期", "最後配種精液", "配種次數", "前次分娩日期", "第一次配種日期", "第一次配種精液"])
    df_birth = pd.read_csv('/content/drive/MyDrive/DL_final/data/birth.csv', header=0, low_memory=False,
                           names=["乳牛編號", "分娩日期", "乾乳日期", "犢牛編號1", "讀牛編號2",
                                  "母牛體重", "登錄日期", "計算胎次", "胎次(birth)", "分娩難易度",
                                  "犢牛體型", "犢牛性別", "酪農場代號"])
    df_breed = pd.read_csv('/content/drive/MyDrive/DL_final/data/breed.csv', header=0, low_memory=False,
                           names=["乳牛編號", "配種日期", "配種精液", "登錄日期", "孕檢",
                                  "配種方式", "精液種類", "酪農場代號"])
    df_spec = pd.read_csv('/content/drive/MyDrive/DL_final/data/spec.csv', header=0, low_memory=False,
                          names=["乳牛編號", "狀況類別", "狀況代號", "狀況日期", "備註",
                                 "登錄日期", "酪農場代號"])
    df_submission = pd.read_csv('/content/drive/MyDrive/DL_final/data/submission.csv', header=0, low_memory=False)

    print("df_report shape = ", df_report.shape)
    print("df_birth shape = ", df_birth.shape)
    print("df_breed shape = ", df_breed.shape)
    print("df_spec shape = ", df_spec.shape)
    print("df_submission shape = ", df_submission.shape)

    df_breed['配種日期'] = pd.to_datetime(df_breed['配種日期'])
    df_breed = df_breed.sort_values(by='配種日期')
    df_breed.reset_index(inplace=True)
    df_spec['狀況日期'] = pd.to_datetime(df_spec['狀況日期'])
    df_spec = df_spec.sort_values(by='狀況日期')
    df_spec.reset_index(inplace=True)

    # create feature table
    birth_feature = pd.DataFrame(columns=["乾乳日期", "母牛體重", "計算胎次", "胎次(birth)", "分娩難易度", "犢牛體型", "犢牛性別"],
                                 index=df_report.index)
    breed_feature = pd.DataFrame(columns=["配種日期", "配種精液", "孕檢", "配種方式", "精液種類"],
                                 index=df_report.index)
    spec_feature = pd.DataFrame(columns=["狀況類別", "狀況代號", "狀況日期"],
                                index=df_report.index)

    for idx in tqdm(range(0, len(df_report))):
        # ---------------------------------birth---------------------------------
        mask1 = df_birth['乳牛編號'] == df_report.at[idx, '乳牛編號']
        mask2 = df_birth['分娩日期'] == df_report.at[idx, '最近分娩日期']
        a = df_birth[mask1 & mask2]
        if not df_birth[mask1 & mask2].empty:
            birth_feature.loc[df_report.index[idx], ["乾乳日期", "母牛體重", "計算胎次", "胎次(birth)", "分娩難易度", "犢牛體型", "犢牛性別"]] = \
                a.iloc[0, 2], a.iloc[0, 5], a.iloc[0, 7], a.iloc[0, 8], a.iloc[0, 9], a.iloc[0, 10], a.iloc[0, 11]
        else:
            birth_feature.loc[df_report.index[idx], ["乾乳日期", "母牛體重", "計算胎次", "胎次(birth)", "分娩難易度", "犢牛體型", "犢牛性別"]] = \
                np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
        # ---------------------------------breed---------------------------------
        mask3 = df_breed['乳牛編號'] == df_report.at[idx, '乳牛編號']
        breed = df_breed[mask3]['配種日期']
        nan = True
        for i in range(0, len(breed)):
            if (pd.to_datetime(breed.iloc[i]) >= pd.to_datetime(df_report.at[idx, '第一次配種日期'])) & \
                    (pd.to_datetime(breed.iloc[i]) <= (pd.to_datetime(df_report.at[idx, '採樣日期']) + timedelta(days=15))):
                breed_feature.loc[df_report.index[idx], ["配種日期", "配種精液", "孕檢", "配種方式", "精液種類"]] = \
                    df_breed[mask3].iloc[i, 2], df_breed[mask3].iloc[i, 3], df_breed[mask3].iloc[i, 5], \
                    df_breed[mask3].iloc[i, 6], df_breed[mask3].iloc[i, 7]
                nan = False
        if nan:
            breed_feature.loc[df_report.index[idx], ["配種日期", "配種精液", "孕檢", "配種方式", "精液種類"]] = \
                np.nan, np.nan, np.nan, np.nan, np.nan
        # ---------------------------------spec---------------------------------
        mask4 = df_spec['乳牛編號'] == df_report.at[idx, '乳牛編號']
        spec = df_spec[mask4]['狀況日期']
        nan = True
        for j in range(0, len(spec)):
            if (pd.to_datetime(spec.iloc[j]) > pd.to_datetime(df_report.at[idx, '最近分娩日期'])) & \
                    (pd.to_datetime(spec.iloc[j]) <= (pd.to_datetime(df_report.at[idx, '採樣日期']) + timedelta(days=30))):
                spec_feature.loc[df_report.index[idx], ["狀況類別", "狀況代號", "狀況日期"]] = \
                    df_spec[mask4].iloc[j, 2], df_spec[mask4].iloc[j, 3], df_spec[mask4].iloc[j, 4]
                nan = False
        if nan:
            spec_feature.loc[df_report.index[idx], ["狀況類別", "狀況代號", "狀況日期"]] = np.nan, np.nan, np.nan

    print(birth_feature.shape)
    print(breed_feature.shape)
    print(spec_feature.shape)

    df_report = df_report.drop('檢測日期', axis=1)
    FeatureTable = pd.concat([df_report, birth_feature], axis=1, join='inner')
    FeatureTable = pd.concat([FeatureTable, breed_feature], axis=1, join='inner')
    FeatureTable = pd.concat([FeatureTable, spec_feature], axis=1, join='inner')
    print(FeatureTable.shape)
    return FeatureTable


# ----------------------------------------------將submission的資料拆分出來----------------------------------------------
def SplitSubmission():
    df_feature = pd.read_csv('/content/drive/MyDrive/DL_final/processed/MergeTable.csv', header=0, low_memory=False)
    df_submission = pd.read_csv('/content/drive/MyDrive/DL_final/data/submission.csv', header=0, low_memory=False)
    df_feature = OneHot(df_feature)
    for i in range(0, 5):
        df_feature = df_feature.sample(frac=1, replace=False)
    x_feature, y_feature, df_feature = preprocessor(df_feature)
    submission_dataset = pd.merge(df_feature, df_submission, on='ID', how='inner')
    submission_dataset = submission_dataset.drop('1', axis=1)
    df_feature = df_feature.dropna(subset=['乳量'])
    # df_feature = OneHot(df_feature)
    # submission_dataset = OneHot(submission_dataset)
    print(df_feature.shape)
    print(submission_dataset.shape)

    x_train = df_feature.drop('乳量', axis=1)
    y_train = df_feature[['ID', '乳量']]
    x_test = submission_dataset.drop('乳量', axis=1)
    y_test = submission_dataset[['ID', '乳量']]
    return x_train, y_train, x_test, y_test


# -------------------------------------------------------前處理-------------------------------------------------------
def checkNaN(df, fill):
    # print(df.name + ' nan = ', df.isna().sum())
    if df.isna().sum() != 0:
        if fill is not None:
            df = df.fillna(fill, inplace=True)
        else:
            df = df.fillna(df.mean(), inplace=True)
        # print(df.name + ' 補值後nan = ', df.isna().sum())
    return df


def standardization(df):
    df = (df - df.mean()) / df.std()
    return df


def preprocessor(df_feature):
    #df_feature = df_feature.sort_values(['資料年度', '資料月份'])
    # 1. 資料年度 drop (train 和 test沒有重疊)
    df_feature = df_feature.drop('資料年度', axis=1)
    #df_feature['資料年度'] = checkNaN(df_feature['資料年度'], fill=None)
    #df_feature['資料年度'] = standardization(df_feature['資料年度'])

    # 2. 資料月份 Standardization
    df_feature['資料月份'] = checkNaN(df_feature['資料月份'], fill=None)
    df_feature['資料月份'] = standardization(df_feature['資料月份'])

    # 8. 胎次 Standardization
    df_feature['胎次'] = checkNaN(df_feature['胎次'], fill=None)
    df_feature['胎次'] = standardization(df_feature['胎次'])

    # 9. 泌乳天數 Standardization
    df_feature['泌乳天數'] = checkNaN(df_feature['泌乳天數'], fill=None)
    df_feature['泌乳天數'] = standardization(df_feature['泌乳天數'])

    # 10. 分娩間隔 (最近分娩日期 - 前次分娩日期) Standardization
    df_feature['分娩間隔'] = pd.to_datetime(df_feature['最近分娩日期']) - pd.to_datetime(df_feature['前次分娩日期'])
    df_feature['分娩間隔'] = checkNaN(df_feature['分娩間隔'], fill='0')
    df_feature['分娩間隔'] = standardization(df_feature['分娩間隔'])

    # 11. 距離最近分娩天數 (採樣日期 - 最近分娩日期) Standardization
    df_feature['距離最近分娩天數'] = pd.to_datetime(df_feature['採樣日期']) - pd.to_datetime(df_feature['最近分娩日期'])
    df_feature['距離最近分娩天數'] = checkNaN(df_feature['距離最近分娩天數'], fill=None)
    df_feature['距離最近分娩天數'] = standardization(df_feature['距離最近分娩天數'])

    # 12. 月齡 Standardization
    df_feature['月齡'] = checkNaN(df_feature['月齡'], fill=None)
    df_feature['月齡'] = standardization(df_feature['月齡'])

    # 13. 配種總時長 (最後配種日期 - 第一次配種日期) Standardization
    df_feature['配種總時長'] = pd.to_datetime(df_feature['最後配種日期']) - pd.to_datetime(df_feature['第一次配種日期'])
    df_feature['配種總時長'] = checkNaN(df_feature['配種總時長'], fill=None)
    df_feature['配種總時長'] = standardization(df_feature['配種總時長'])

    # 15. 配種次數 Standardization
    df_feature['配種次數'] = checkNaN(df_feature['配種次數'], fill=None)
    df_feature['配種次數'] = standardization(df_feature['配種次數'])

    # 16. 前次分娩日期 drop
    df_feature = df_feature.drop('前次分娩日期', axis=1)

    # 17. 分娩後重新配種 (第一次配種日期 - 最近分娩日期) Standardization
    df_feature['分娩後重新配種'] = pd.to_datetime(df_feature['第一次配種日期']) - pd.to_datetime(df_feature['最近分娩日期'])
    df_feature['分娩後重新配種'] = checkNaN(df_feature['分娩後重新配種'], fill=None)
    df_feature['分娩後重新配種'] = standardization(df_feature['分娩後重新配種'])

    # 19. 乾乳天數 (乾乳日期 - 最近分娩日期) Standardization
    #df_feature['乾乳天數'] = pd.to_datetime(df_feature['乾乳日期']) - pd.to_datetime(df_feature['最近分娩日期'])
    #df_feature['乾乳天數'] = checkNaN(df_feature['乾乳天數'], fill=None)
    #df_feature['乾乳天數'] = standardization(df_feature['乾乳天數'])

    # 20. 母牛體重 Standardization
    # df_feature = df_feature.drop('母牛體重', axis=1)
    df_feature['母牛體重'] = checkNaN(df_feature['母牛體重'], fill=None)
    df_feature['母牛體重'] = standardization(df_feature['母牛體重'])

    # 21. 計算胎次 drop
    df_feature = df_feature.drop('計算胎次', axis=1)

    # 22. 胎次(birth)
    df_feature = df_feature.drop('胎次(birth)', axis=1)
    #df_feature['胎次(birth)'] = checkNaN(df_feature['胎次(birth)'], fill=None)
    #df_feature['胎次(birth)'] = standardization(df_feature['胎次(birth)'])
    
    # 23. 分娩難易度 Standardization
    df_feature['分娩難易度'] = checkNaN(df_feature['分娩難易度'], fill=1)
    df_feature['分娩難易度'] = standardization(df_feature['分娩難易度'])

    # 24. 犢牛體型 drop (test 都沒M沒有意義)
    df_feature = df_feature.drop('犢牛體型', axis=1)
    # df_feature['犢牛體型'] = checkNaN(df_feature['犢牛體型'], fill='M')
    # sizeDict = {'S': 1, 'M': 2, 'L': 3}
    # df_feature['犢牛體型'] = df_feature['犢牛體型'].map(sizeDict)
    # df_feature['犢牛體型'] = standardization(df_feature['犢牛體型'])

    # 26. 配種間隔 (配種日期 - 第一次配種日期) standardization
    df_feature['配種間隔'] = pd.to_datetime(df_feature['配種日期']) - pd.to_datetime(df_feature['第一次配種日期'])
    df_feature['配種間隔'] = checkNaN(df_feature['配種間隔'], fill=None)
    df_feature['配種間隔'] = standardization(df_feature['配種間隔'])

    # 28. drop 孕檢
    df_feature = df_feature.drop('孕檢', axis=1)

    # 29. 配種方式 drop(缺太多)
    df_feature = df_feature.drop('配種方式', axis=1)

    # 30. 精液種類 drop(缺太多)
    df_feature = df_feature.drop('精液種類', axis=1)

    # 31. 狀況類別 (N:1, C:2, D:3), 其他補0
    df_feature['狀況類別'] = checkNaN(df_feature['狀況類別'], fill='none')
    typeDict = {'N': 1, 'C': 2, 'D': 3, 'none': 0}
    df_feature['狀況類別'] = df_feature['狀況類別'].map(typeDict)
    df_feature['狀況類別'] = standardization(df_feature['狀況類別'])

    # 33. 狀況日期
    df_feature['狀況日期'] = pd.to_datetime(df_feature['狀況日期']) - pd.to_datetime(df_feature['採樣日期'])
    df_feature['狀況日期'] = checkNaN(df_feature['狀況日期'], fill='0')
    df_feature['狀況日期'] = standardization(df_feature['狀況日期'])

    # 34. drop
    df_feature = df_feature.drop('最近分娩日期', axis=1)
    df_feature = df_feature.drop('採樣日期', axis=1)
    df_feature = df_feature.drop('最後配種日期', axis=1)
    df_feature = df_feature.drop('第一次配種日期', axis=1)
    df_feature = df_feature.drop('乾乳日期', axis=1)
    df_feature = df_feature.drop('配種日期', axis=1)
    df_feature = df_feature.drop('狀況日期', axis=1)

    z = df_feature
    x = df_feature.drop('乳量', axis=1)
    y = df_feature[['ID', '乳量']]

    return x, y, z


def OneHot(df_feature):
    # 3. 酪農場代號 one hot
    df_feature['酪農場代號'] = checkNaN(df_feature['酪農場代號'], fill='unknow')
    farm_onehot = pd.get_dummies(df_feature['酪農場代號'], prefix='酪農場代號')
    df_feature = df_feature.drop('酪農場代號', axis=1)
    df_feature = pd.concat([df_feature, farm_onehot], axis=1)

    # 4. 乳牛編號 one hot
    # df_feature = df_feature.drop('乳牛編號', axis=1)
    df_feature['乳牛編號'] = checkNaN(df_feature['乳牛編號'], fill='unknow')
    cow_onehot = pd.get_dummies(df_feature['乳牛編號'], prefix='乳牛編號')
    df_feature = df_feature.drop('乳牛編號', axis=1)
    df_feature = pd.concat([df_feature, cow_onehot], axis=1)


    # 5. 父親精液 one hot
    df_feature['父親牛精液編號'] = checkNaN(df_feature['父親牛精液編號'], fill='unknow')
    dad_onehot = pd.get_dummies(df_feature['父親牛精液編號'], prefix='父親牛精液編號')
    df_feature = df_feature.drop('父親牛精液編號', axis=1)
    df_feature = pd.concat([df_feature, dad_onehot], axis=1)

    # 6. 母親乳牛 one hot
    df_feature['母親乳牛編號'] = checkNaN(df_feature['母親乳牛編號'], fill='unknow')
    mom_onehot = pd.get_dummies(df_feature['母親乳牛編號'], prefix='母親乳牛編號')
    df_feature = df_feature.drop('母親乳牛編號', axis=1)
    df_feature = pd.concat([df_feature, mom_onehot], axis=1)

    # 7. 出生日期 -> 季節 -> one hot
    seasonDict = {3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer',
                  9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter', 1: 'Winter', 2: 'Winter'}
    df_feature['出生日期'] = pd.to_datetime(df_feature['出生日期']).dt.month
    df_feature['出生日期'] = df_feature['出生日期'].map(seasonDict)
    season_onehot = pd.get_dummies(df_feature['出生日期'], prefix='出生日期')
    df_feature = df_feature.drop('出生日期', axis=1)
    df_feature = pd.concat([df_feature, season_onehot], axis=1)

    # 14. 最後配種精液 one hot
    # df_feature = df_feature.drop('最後配種精液', axis=1)
    df_feature['最後配種精液'] = checkNaN(df_feature['最後配種精液'], fill='unknow')
    last_onehot = pd.get_dummies(df_feature['最後配種精液'], prefix='最後配種精液')
    df_feature = df_feature.drop('最後配種精液', axis=1)
    df_feature = pd.concat([df_feature, last_onehot], axis=1)

    # 18. 第一次配種精液 one hot
    # df_feature = df_feature.drop('第一次配種精液', axis=1)
    df_feature['第一次配種精液'] = checkNaN(df_feature['第一次配種精液'], fill='unknow')
    last_onehot = pd.get_dummies(df_feature['第一次配種精液'], prefix='第一次配種精液')
    df_feature = df_feature.drop('第一次配種精液', axis=1)
    df_feature = pd.concat([df_feature, last_onehot], axis=1)

    # 25. 犢牛性別  one hot (缺24991 感覺可以drop)
    df_feature = df_feature.drop('犢牛性別', axis=1)
    #df_feature['犢牛性別'] = checkNaN(df_feature['犢牛性別'], fill='unknow')
    #gender_onehot = pd.get_dummies(df_feature['犢牛性別'], prefix='犢牛性別')
    #df_feature = df_feature.drop('犢牛性別', axis=1)
    #df_feature = pd.concat([df_feature, gender_onehot], axis=1)
    

    # 27. 配種精液 one hot
    df_feature = df_feature.drop('配種精液', axis=1)
    #df_feature['配種精液'] = checkNaN(df_feature['配種精液'], fill='unknow')
    #breed_onehot = pd.get_dummies(df_feature['配種精液'], prefix='配種精液')
    #df_feature = df_feature.drop('配種精液', axis=1)
    #df_feature = pd.concat([df_feature, breed_onehot], axis=1)

    # 32. 狀況代號 one hot
    df_feature = df_feature.drop('狀況代號', axis=1)
    #df_feature['狀況代號'] = checkNaN(df_feature['狀況代號'], fill='nono')
    #type_onehot = pd.get_dummies(df_feature['狀況代號'], prefix='狀況代號')
    #df_feature = df_feature.drop('狀況代號', axis=1)
    #df_feature = pd.concat([df_feature, type_onehot], axis=1)
    return df_feature


# 合併表後匯出
output('/content/drive/MyDrive/DL_final/processed/MergeTable.csv', merge())


# 將submission拆分
x_train, y_train, x_test, y_test = SplitSubmission()

print('x_train.shape= {}, y_train.shape= {}'.format(x_train.shape, y_train.shape))
print('x_test.shape= {}, y_test.shape= {}'.format(x_test.shape, y_test.shape))

output('/content/drive/MyDrive/DL_final/processed/x_train.csv', x_train)
output('/content/drive/MyDrive/DL_final/processed/y_train.csv', y_train)
output('/content/drive/MyDrive/DL_final/processed/x_test.csv', x_test)
output('/content/drive/MyDrive/DL_final/processed/y_test.csv', y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DL_final
(33254, 2013)
(4263, 2013)
x_train.shape= (33254, 2012), y_train.shape= (33254, 2)
x_test.shape= (4263, 2012), y_test.shape= (4263, 2)
Success output to /content/drive/MyDrive/DL_final/processed/x_train.csv
Success output to /content/drive/MyDrive/DL_final/processed/y_train.csv
Success output to /content/drive/MyDrive/DL_final/processed/x_test.csv
Success output to /content/drive/MyDrive/DL_final/processed/y_test.csv
